In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import missingno
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
# Импортируем датасет
df = pd.read_csv('../input/imdb-india-movies/IMDb Movies India.csv')

In [ ]:
# Выведем наш датасет
df

In [ ]:
# Видим очень много пропусков, попробуем решить
df.info()

In [ ]:
# Визуализируем пропущенные значения
missingno.bar(df)

In [ ]:
# Приводим некоторые колонки к удобному названию
df.rename(columns={'Actor 1':'Actor_1','Actor 2':'Actor_2','Actor 3':'Actor_3'}, inplace=True)

In [ ]:
# Чистим строчки, где пропущены данные хотя-бы в одном столбце, кроме актёров
df = df.dropna(subset=['Name', "Genre", "Rating", "Votes", "Director"]).reset_index(drop=True)

In [ ]:
# Видим, что отсеялось почти 10 тысяч строк
df

In [ ]:
# Видим, что пропусков нигде, кроме актёров нет, а данные таких полей, как [Year,Duration,Votes] не числовые. Исправим всё разом
df.info()

In [ ]:
# Приведём все три колонки [Year,Duration,Votes] к числовому типу и заменим значения, где отсутсвуют актёры на "None"
for index, row in df.iterrows():
    df.at[index, "Votes"] = pd.to_numeric(row["Votes"].replace(',',''), errors='coerce')
    df.at[index, "Year"] = pd.to_numeric(row["Year"].replace("(",'').replace(")",''), errors='coerce')
    if not pd.isna(row["Duration"]):
        df.at[index, "Duration"] = pd.to_numeric(row["Duration"].replace(' min',''), errors='coerce') 
    for id in range(1,4):
        if pd.isna(row["Actor_"+str(id)]): 
            df.at[index,"Actor_"+str(id)] = "None" 
df = df.astype({"Votes": "Int64", "Year": "Int64", "Duration": "Int64"})

In [ ]:
# Приведение типов выполнено успешно, пропусков нет
df['Duration'].fillna(df['Duration'].median(), inplace=True)
df.info()

In [ ]:
# Воспользовавшись принципом дамми-кодирования преобразуем нашу таблицу
dummies = df['Genre'].str.get_dummies(sep=', ')
df_genre = pd.concat([df, dummies], axis=1)

# И посмотрим сколько тех или иных фильмов по жанрам было выпущено
df_genre.drop(['Name','Duration','Year','Genre','Rating','Votes','Director','Actor_1','Actor_2','Actor_3'], axis=1).sum().sort_values(ascending=False)

In [ ]:
df.describe()

In [ ]:
plt.figure(figsize=(13,13))
sns.heatmap(df.corr(method='spearman'),annot=True,cmap="Blues", fmt='.0%')
plt.show()

In [ ]:
fig = px.scatter_matrix(df,width=2500, height=2500)
fig.show()

In [ ]:
# Избавимся от лишних колонок
df_genre_count = df_genre.drop(['Name','Duration','Genre','Rating','Votes','Director','Actor_1','Actor_2','Actor_3'], axis=1)
df_genre_count = df_genre_count.groupby('Year').sum()

# Поставим индексом годы
df_genre_count.reset_index(level=0, inplace=True)
df_genre_count.index = df_genre_count["Year"]
df_genre_count = df_genre_count.drop(["Year"],axis = 1)


In [ ]:
# И выведем график на преобразованном датасете, который покажет нам фильмы по жанрам выпщенные в то или иное время
fig = px.line(df_genre_count, x = df_genre_count.index, y = df_genre_count.columns,labels={
                     "value": "Количество выпущенных фильмов",
                     "Year": "Год",
                     "variable": "Жанры"
                 },
                title="Пожанровая таблица выпущенных фильмов за всё время")
fig.show()

In [ ]:
df_directors = df['Director'].value_counts()

fig = go.Figure(go.Bar(x = df_directors.index[0:10], y = df_directors.values[0:10]))
fig.update_layout(
    title='10 самых частых режиссёров',
    height = 500)
fig.show()



In [ ]:
# Для того, чтобы вывести топ 30 актёров, нужно произвести конкатенацию 3 матриц, однако тут 3 столбца. Преобразуем их в матрицы
Actor1 = df[['Actor_1','Year']].rename(columns = {'Actor_1': 'Actor'}, inplace = False)
Actor2 = df[['Actor_2','Year']].rename(columns = {'Actor_2': 'Actor'}, inplace = False)
Actor3 = df[['Actor_3','Year']].rename(columns = {'Actor_3': 'Actor'}, inplace = False)

Actor_Year = pd.concat([Actor1, Actor2, Actor3], ignore_index=True).dropna()
Actor_Top = Actor_Year['Actor'].value_counts().rename_axis('Actor').reset_index(name='Count')
Actor_Top = Actor_Top.drop(index=[0]) # Удаляем пустых("None") 

fig = go.Figure(go.Bar(x = Actor_Top["Actor"][0:10], y = Actor_Top["Count"][0:10]))
fig.update_layout(
    title='10 самых частых актёров',
    height = 500)
fig.show()


In [ ]:
best_by_rating = df.sort_values("Rating", ascending=False).head(10)[[ "Name", "Year", "Duration", "Votes", "Rating", "Director","Actor_1","Actor_2","Actor_3"]].reset_index(drop=True)
fig = ff.create_table(best_by_rating)
fig.update_layout(
    title_text = 'Топ 10 фильмов по рейтингу',
    height = 600,
    margin = {'t':50},)
fig.show()

In [ ]:
best_by_votes = df.sort_values("Votes", ascending=False).head(10)[[ "Name", "Year", "Duration", "Votes", "Rating", "Director","Actor_1","Actor_2","Actor_3"]].reset_index(drop=True)
fig = ff.create_table(best_by_votes)
fig.update_layout(
    title_text = 'Топ 10 фильмов по количеству голосов',
    height = 600,
    margin = {'t':50},)
fig.show()

In [ ]:
# Извлекаем зависимые 
rating = df.pop('Rating')
votes = df.pop('Votes')
y = pd.DataFrame(data = [rating,votes]).T
y